In [ ]:
 # Need to install from github, for staying up-to-date with the latest developments.
  # reason: if a bug has been fixed since the last official release but a new release hasn’t been rolled out yet
%pip -q install git+https://github.com/huggingface/transformers
# %pip -q install transformers
%pip -q install -U datasets
%pip -q install -U loralib
%pip -q install -U sentencepiece
%pip -q install -U bitsandbytes
%pip -q install -U accelerate
%pip -q install -U einops
%pip -q install -U langchain
%pip -q install -U huggingface_hub
%pip -q install -U chromadb
%pip -q install -U PyPDF2
%pip -q install -U pypdf
%pip -q install -U sentence-transformers
%pip -q install -U FlagEmbedding
%pip -q install -U InstructorEmbedding

# %pip -q install xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# !nvidia-smi

###Download the PDFs as external resourses

This part show that we can load a link and extract many pdf files, even the ones with misinformation included

In [ ]:
!wget -O PDFPapers.zip your_path_to_the_zip_file
!unzip -q PDFPapers.zip -d papers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
# import os
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# 'mixedbread-ai/mxbai-embed-large-v1' embedding
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
# import json
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
# Tokenizer that correspond to the model, used to convert text to a fromat that model can understand(tokenization) and back to the text(detokenization)
base_model_name = "nmdr/llama-2-7b-chat-platypus-1k-Baymax10"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                            #  use_auth_token = True,
                                             load_in_8bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:758: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access pub

tokenizer_config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

###Building Pipeline

“Max Length” controls the overall length of the response.(restricts the total length (input + output))

“Max New Tokens” specifically limits the tokens generated beyond the input. It ensures that the output aligns with your desired length while considering the context provided.(specifically limits the tokens generated beyond the input)


https://medium.com/@developer.yasir.pk/understanding-the-controllable-parameters-to-run-inference-your-large-language-model-**30643bb46434**

In [ ]:
# To create a text generation pipeline

# pipelie(): The pipeline is a high-level utility that simplifies the usage of Transformer models for various tasks, such as text generation
# do_sample: Enables sampling, this allows the model to generate text probabilistically rather than deterministically. Sampling can lead to more varied and interesting outputs
# top_k: Sample from the top k most likely next tokens at each step, This helps in reducing the randomness of the output, providing a balance between creativity and coherence
# eos_token_id: specify the token that indicates the end of a sequence, Allowing the model to determine when to stop generating further tokens

# "text-classification"
pipe = pipeline("text-generation", # specify the task for the pipeline
               model = model,
               tokenizer = tokenizer,
              #  torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                max_new_tokens=512,
              #  device_map = 'auto',
              #  do_sample = True,
              #  top_k = 30,
              #  num_return_sequences = 1, # only one text sequence should be return for each input
              #  eos_token_id = tokenizer.eos_token_id
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# print(hf_llm("Who are you?"))
# pipe("Who are you?")

# sequences = pipe("Who are you")
# for seq in sequences:
#   print(f"reuslts: {seq['generated_text']}")

# pipe("I'm in a good mood")

In [ ]:
# tokenzier.vocab_size
# tokenizer.all_special_tokens
# tokenizer.all_special_ids
# tokenizer(['<unk>'])
# tokenizer(['<SYS>\n'])
# tokenizer.decode([1, 14816, 29903, 6778, 13]) # output: '<s>SYS>>\n'

###Setting up Langchain to retrieve PDFs

Load and process PDFs

In [ ]:
# Load and process a single text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

360

In [ ]:
# splitting the text into chunks
# chunk_overlap: if we get one idea between two chunks of text,we want it to be overlapped, so we can actually get that in one full chunk by itself.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1324

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

###BAAI/bge-large-en-v1.5

In [ ]:
# HuggingFace Embeddings - Instructor embeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
#                                                       model_kwargs={"device": "cuda"})


embedding_model_name = "BAAI/bge-base-en-v1.5"
# model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###mixedbread-ai/mxbai-embed-large-v1

note that you have to provide the prompt "Represent this sentence for searching relevant passages: "
for query if you want to use it for retrieval. Besides that you don't need any prompt.

In [ ]:
# # loading model
# model_name = 'mixedbread-ai/mxbai-embed-large-v1'

# # Encoding
# # encode_kwargs = {'normalized': True}

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     # encode_kwargs=encode_kwargs
# )

###Chroma DB

Chroma DB is a vector store that is open-source and is utilized for the storage and retrieval of vector embeddings. Its primary purpose is to store embeddings and associated metadata for future use by extensive language models. Furthermore, it can also be employed for semantic search engines that operate on text data.

With Chroma DB, you can easily manage text documents, convert text to embeddings, and do similarity searches.

https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever

vectordb:
This appears to be a reference to a module or an object that interacts with a vector database system. Vector databases are specialized storage systems designed to handle high-dimensional vector data efficiently, which is common in machine learning and similar applications where entities are represented as vectors in a high-dimensional space.

as_retriever:
as_retriever is a method that configures and returns a retriever object. This object is likely used for querying the vector database, particularly for retrieving vectors that are nearest to a given query vector based on some distance metric (e.g., cosine similarity).

search_kwargs:
The search_kwargs parameter is used to pass additional keyword arguments to the as_retriever method. These arguments are typically used to configure how the search is performed within the vector database.

{"k": 5}: This dictionary specifies the configuration for the retriever. Here, k is set to 5, which usually means the retriever will return the top 5 closest vectors (or entries) in response to a query. This is often referred to as "k-nearest neighbors" search.

In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

###LangChain
LangChain components can be used to build new prompt chains or customize existing templates. it also also includes components that allow LLMs to access new data sets without retraining.

LLMs excel at responding to prompts in a general context, but struggle in a specific domain they were never trained on. Prompts are queries people use to seek responses from an LLM. For example, an LLM can provide an answer to how much a computer costs by providing an estimate. However, it can't list the price of a specific computer model that your company sells.

To do that, machine learning engineers must integrate the LLM with the organization’s internal data sources and apply prompt engineering*—a practice where a data scientist refines inputs to a generative model with a specific structure and context.

LangChain streamlines intermediate steps to develop such data-responsive applications, making prompt engineering more efficient. It is designed to develop diverse applications powered by language models more effortlessly, including chatbots, question-answering, content generation, summarizers, and more.[https://aws.amazon.com/what-is/langchain/]

*Prompt engineering is the process where you guide generative artificial intelligence (generative AI) solutions to generate desired outputs.[https://aws.amazon.com/what-is/prompt-engineering/]



In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# DEFAULT_SYSTEM_PROMPT = """\
# You are a drunk, not respectful assistant. Always answer with creativity. Your answers could include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are biased and negative in nature."""


# Creates a complete prompt
def get_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
  sys_prompt = B_SYS + system_prompt + E_SYS
  prompt_template = B_INST + sys_prompt + instruction + E_INST
  # print(prompt_template)
  return prompt_template


###Building a new system propmt

In [ ]:
# instruction = "Summarize the following text for me {text}"

# system_propmt = "Your are an expert in text and article summarization and reducing the number of words. All the sentences and the grammar should be academically enhanced by you."

# get_prompt(inst)

###Building new system prompts

In [ ]:
# system_prompt = "You are an expert assistant in translation."
# instruction = "Convert the text from English to Italian:\n\n {text}"
# prompt_template = get_prompt(instruction, system_prompt)
# print(prompt_template)

# prompt = PromptTemplate(template=prompt_template, input_variable=["text"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

### Completely different system prompt

In [ ]:
# diffrent system propmt
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

#mxbai syetem prompt
# system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: Represent this sentence for searching relevant passages: {question}"""
# get_prompt(instruction, sys_prompt)

### RetrievalQA

Chain Type

The default chain_type="stuff" uses ALL of the text from the documents in the prompt. It actually doesn’t work with our example because it exceeds the token limit and causes rate-limiting errors. That’s why in this example, we had to use other chain types for example "map_reduce". What are the other chain types?

map_reduce: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.

refine : It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.

map-rerank: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [ ]:
# Create the template prompt
prompt_template = get_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )


# llm_chain = LLMChain(prompt=prompt, llm=llm)
chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses ALL of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# to format the response and cite sources
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

# To trim a given string (text) at the point where a specific substring (prompt) first appears
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:] # Trim everything from the start of text up to the phrase/symbol
  else:
    return output_text


# Removes occurrences of a substring from a string, typically used here to clean up the generated text by removing predefined markers or prompts
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    # source_list = []
    # print(wrap_text(generated_response['result']))
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '[/INST]')
    final_response = remove_substring(final_response, '[/INST]')
    print(final_response)

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      # source_list.append(source.metadata['source'])
      print(source.metadata['source'])

    return final_response

In [ ]:
# qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
# For retrieval we need to pass this prompt.
# query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# # process_generated_response(response)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# query = "How many tokens was LLaMA-2 trained on?"
# response = qa_chain(query)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# # For retrieval we need to pass this prompt.
# # query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# process_generated_response(response)

In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  LLaMA-2 was trained on approximately 2 trillion tokens.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
  LLaMA-2 was trained on approximately 2 trillion tokens.


###retrieving questions and generating responses

In [ ]:
# %pip -q install dropbox

# import pathlib
# import pandas as pd
# import dropbox
# from dropbox.exceptions import AuthError

# DROPBOX_ACCESS_TOKEN = ''

# # Connect to the Dropbox API
# def dropbox_connect():
#   try:
#     dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
#   except AuthError as e:
#     print(f"Error connecting to Dropbox with access token: {str(e)}" )
#   return dbx


# # Download the file
# def dropbox_download(dbx_file_path, local_file_path):
#   try:
#     dbx = dropbox_connect()

#     with open(local_file_path, 'wt') as f:
#       metadata, result = dbx.files_download(path=dbx_file_path)
#       f.write(result.content)
#   except Exception as e:
#       print(f"Error downloading file from dropbox: {str(e)}")

# dbx_path_file = 'All files/Apps/LLMs-RAG/Questions.csv'
# local_path_file = '/content/Questions'
# Questions = dropbox_download(dbx_path_file, local_path_file)

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  # dfQ
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  # print("These are the General Questions: \n")
  # # print(f"{dfQ.loc[:, 'Question']}")
  # for index in range(len(dfQ)):
  #   print(f"Q {index+1}: {dfQ.loc[index,'Question']}")
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    # print(str(k))
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


# Download the Questions in CSV format

In [ ]:
!wget -O Questionscsv.zip your_path_to_the_zip_file
!unzip -q Questionscsv.zip -d questions

###General QA

In [ ]:
# General Questions

questions_file_path = '/content/questions/Questions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Newton's First Law of Motion states that an object at rest remains at rest, and an object in
motion remains in motion unless acted upon by external forces.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The force of gravity between two objects changes with
distance according to Newton's Law of Universal Gravitation, which states that the force of gravity between two objects is proportional to the product
of their masses and inversely proportional to the square of the distance between them. Mathematically, this can be expressed as:

F = G \* (m_1 \* m_2) / r^2

where F is the force of gravity between the two objects, G is the gravitational constant (approximately 6.67 x 10^-11 N\*m^2/kg^2), m_1 and m_2 are
the masses of the two objects, and r is the distance between them.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Activation functions play a crucial role in neural networks by
introducing nonlinearity into the model's decision boundary. They transform the output of each neuron into a higher-dimensional space, allowing the
network to learn complex relationships between inputs and outputs. Common activation functions used in deep learning include sigmoid, tanh, ReLU
(Rectified Linear Unit), and its variants like LeakyReLU and ParametricReLU. These functions have different properties, such as zero-centeredness,
range, and slope, which determine how they behave when passing through the origin or at extreme values. Understanding the behavior of activation
functions is essential for designing efficient and effective neural network architectures.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Supervised learning involves training a
machine learning algorithm on labeled data, where the target output is already known. The goal is to learn a mapping between input features and
corresponding outputs so that new inputs can be predicted accurately. Unsupervised learning, on the other hand, focuses on discovering patterns or
relationships within the data without prior knowledge of the target output. Instead, algorithms aim to identify clusters, dimensions, or anomalies in
the data itself. While both types of learning have their applications, supervised learning tends to be more common due to its practical nature and
potential impact on real-world problems. However, unsupervised learning has seen significant advancements recently, particularly in deep learning
techniques like autoencoders and generative adversarial networks (GANs).


Sources:
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Overfitting refers to a situation where a neural
network learns the noise present in its training data rather than generalizing well to new, unseen data. This occurs when the model becomes too
complex and starts memorizing the training examples rather than learning generalizable patterns. As a result, the model performs poorly on test data
outside the training set. Techniques like regularization, dropout, early stopping, and batch normalization can help prevent overfitting by reducing
the capacity of the model and encouraging it to learn more robust features.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
  The learning rate in training neural networks refers to
how quickly the weights are updated based on the gradient of the loss function. A higher l

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is Newton's first law of motion?
 response:   Newton's First Law of Motion states that an object at rest remains at rest, and an object in
motion remains in motion unless acted upon by external forces.


Q1: How does the force of gravity between two objects change with distance?
 response:   The force of gravity between two objects changes with
distance according to Newton's Law of Universal Gravitation, which states that the force of gravity between two objects is proportional to the product
of their masses and inversely proportional to the square of the distance between them. Mathematically, this can be expressed as:

F = G \* (m_1 \* m_2) / r^2

where F is the force of gravity between the two objects, G is the gravitational constant (approximately 6.67 x 10^-11 N\*m^2/kg^2), m_1 and m_2 are
the masses of the two objects, and r is the distance between them.


Q2: What is the principle of conservation of energy?
 response:   The principle of conservation of energy states that

In [ ]:
json_file_name = 'generalQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'generalQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Cosmology QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroCosmoQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Studying astrophysics and cosmology aim to understand the origin,
evolution, structure, and fate of the universe through observations, experiments, simulations, and theoretical models. These fields seek to unravel
the mysteries of dark matter, dark energy, black holes, galaxy formation, star birth, planetary systems, and other phenomena that shape our cosmic
landscape. By gaining insights into the fundamental laws governing the behavior of matter and energy within the universe, astrophysicists and
cosmologists can develop theories that predict future events and guide further exploration and discovery. Ultimately, understanding the universe
allows us to better appreciate our place within it and contemplate the vastness of existence beyond our small corner of the cosmos.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Augmented Reality in astrophys

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to the text, several factors support the Big Bang theory:

1. Observation of the cosmic microwave background radiation (CMB): This radiation is thought to be remnant heat from the initial explosion. Its
uniformity across the sky suggests that the universe began in a very hot state and expanded rapidly.
2. Abundance of light elements: According to the Big Bang theory, these elements were formed during the first few minutes after the universe began.
Their abundance matches what would be expected if they were created in this way.
3. Large scale structure of the universe: Galaxies and galaxy clusters form a hierarchical structure, with smaller ones merging into larger ones. This
structure is consistent with the idea that the universe started in a very hot state and gradually cooled while expanding.
4. Redshift of light from distant galaxies: The farther away a galaxy is, the faster its light is shifted towards the red end of the spectrum due to
the expansion of space. This obse

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Recent developments in cosmology include the discovery of dark matter and dark
energy, which together account for approximately 95% of the universe's mass-energy budget. Additionally, observations of the cosmic microwave
background radiation have confirmed the Big Bang model and provided insights into the evolution of the universe since its formation. Other notable
advancements include the development of new telescopes such as the Atacama Large Millimeter/submillimeter Array (ALMA), which allow for more detailed
studies of distant galaxies and galaxy clusters, as well as improvements in computational power and data analysis techniques enabling researchers to
analyze larger datasets than ever before.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The observational revolution in cosmology refers to the significant
advancements made in measuring various cosmic phenomena over the past few decades, providing us with more accurate determinations of the standard
cosmological model's parameters. This has been achieved through observations of redshift supernovae, gravitational lensing, large scale structure
measurements, distribution of galaxies, and many other factors. These new data constrain the parameter space of cosmological models, leading to a
greater understanding of the universe's evolution and properties. As a result, cosmology will become increasingly phenomenological, focusing on
interpreting experimental results rather than developing new theories.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Cosmological observations are explained by various theoretical models,
including the Big Bang theory, inflationary theories, dark matter models, and modified gravity theories. The Standard Model of cosmology, based on the
Big Bang theory, has been successful in explaining many observations, but it still faces challenges in reconciling certain aspects of the data with
its predictions. For example, the matter-energy budget of the universe remains uncertain due to the difficulty in detecting dark matter directly.
Additionally, some observations suggest that dark energy may play a role beyond what the Standard Model can account for. Other alternative models,
such as plasma redshift cosmology, have gained attention recently due to their ability to explain certain observations without invoking dark energy or
dark matter. These models propose modifications to Einstein's theory of general relativity to accommodate the observed phenomena. Overall, while the
Standard Model provides a solid fou

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the goal of studying astrophysics and cosmology?
 response:   Studying astrophysics and cosmology aim to understand the origin,
evolution, structure, and fate of the universe through observations, experiments, simulations, and theoretical models. These fields seek to unravel
the mysteries of dark matter, dark energy, black holes, galaxy formation, star birth, planetary systems, and other phenomena that shape our cosmic
landscape. By gaining insights into the fundamental laws governing the behavior of matter and energy within the universe, astrophysicists and
cosmologists can develop theories that predict future events and guide further exploration and discovery. Ultimately, understanding the universe
allows us to better appreciate our place within it and contemplate the vastness of existence beyond our small corner of the cosmos.


Q1: What supports the Big Bang theory?
 response:   According to the text, several factors support the Big Bang theory:

1. Observation of the c

In [ ]:
json_file_name = 'AstroCosmoQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroCosmoQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Physics QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroPhysicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Chandrasekhar limit refers to the maximum mass of a white
dwarf star beyond which it cannot maintain its stability due to the increasing gravitational pressure caused by its own weight. It was first proposed
by Indian astrophysicist Subrahmanyan Chandrasekhar in 1930 and later confirmed through observations of actual white dwarfs. The limit is around 1.4
solar masses (M☉), meaning that any white dwarf with a mass greater than this value will eventually collapse into a neutron star or black hole.
Understanding the Chandrasekhar limit has been crucial in understanding the final stages of a star's life cycle and how different types of stars
evolve over time.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  White dwarf stars play a crucial role in our
understanding of stellar evolution. They represent the final stage of a star's life cycle, after it has exhausted its fuel supply and collapsed into a
dense, hot core. These stars are incredibly small, with diameters ranging from approximately 10,000 to 1 million kilometers, depending on their mass.
Despite their tiny size, they emit intense radiation due to their high surface temperature, making them visible even though they are no longer massive
enough to sustain nuclear fusion reactions.

One key aspect of white dwarf stars is their composition. Because they are composed primarily of carbon and oxygen atoms, they are often referred to
as "ash" or "remnant" stars. This composition leads to unique properties compared to other types of stars, including low luminosity and high surface
gravities. Additionally, white dwarfs can undergo various processes throughout their lives, such as cooling over billions of years until reaching
temperatures

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


Experimental advances that have allowed for the direct observation of gravitational waves include the Laser Interferometer Gravitational-Wave
Observatory (LIGO) and Virgo detectors, which utilize laser interferometry to measure tiny changes in distance caused by passing gravitational waves.
These observations were made possible through significant technological improvements over previous generations of gravitational wave detectors,
including improved sensitivity, better noise reduction techniques, and more sophisticated data analysis methods.

These advances represent a major breakthrough in astrophysics because they provide a new window into understanding cosmic phenomena that were
previously invisible to human observers. For example, gravitational waves emitted by merging black holes or neutron stars can offer insights into
these objects' properties and behavior, shedding light on fundamental physics questions like how matter behaves under extreme conditions.
Additionally, studying 

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The discovery of black holes
challenges our understanding of physics under extreme conditions by providing evidence that certain predictions made by Einstein's theory of general
relativity can occur in nature. Specifically, the existence of black holes suggests that matter can become so dense that its gravitational pull
becomes infinite, leading to the formation of a singularity where space-time ceases to exist. This phenomenon pushes the limits of our current
understanding of how matter behaves under such intense gravitational forces and highlights areas where further research is needed to fully comprehend
the behavior of matter in extreme environments. Additionally, studying black holes allows scientists to test their theories about gravity and the
structure of spacetime, providing valuable insights into fundamental aspects of physics.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selecte

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Contemporary
challenges in theoretical physics suggested by the study of cosmological phenomena include understanding the origins of the universe, developing
accurate models of inflationary cosmology, and determining the parameters of our standard cosmic model through observational evidence. Additionally,
there is ongoing debate about the nature of dark matter and dark energy, which remain unexplained components of the universe.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
  Based on the given text, there are
several future technologies that are anticipated to advance our exploration of cosmological phenomena:

1. Inflationary theory: Developed by Alan Guth in the early 1980s, inflationary theory proposes that the universe underwent a rapid expans

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the Chandrasekhar limit and its significance in astrophysics?
 response:   The Chandrasekhar limit refers to the maximum mass of a white
dwarf star beyond which it cannot maintain its stability due to the increasing gravitational pressure caused by its own weight. It was first proposed
by Indian astrophysicist Subrahmanyan Chandrasekhar in 1930 and later confirmed through observations of actual white dwarfs. The limit is around 1.4
solar masses (M☉), meaning that any white dwarf with a mass greater than this value will eventually collapse into a neutron star or black hole.
Understanding the Chandrasekhar limit has been crucial in understanding the final stages of a star's life cycle and how different types of stars
evolve over time.


Q1: How do white dwarf stars contribute to our understanding of stellar evolution?
 response:   White dwarf stars play a crucial role in our
understanding of stellar evolution. They represent the final stage of a star's life cycle, after it ha

In [ ]:
json_file_name = 'AstroPhysicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroPhysicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

### Attention Questions

In [ ]:
# Attention Questions

questions_file_path = '/content/questions/AttQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer model introduces several innovative features compared to
traditional recurrent neural network (RNN) architectures:

1. Self-Attention Mechanism: Instead of relying solely on recurrence, the Transformer uses a self-attention mechanism to compute the weighted sum of
input elements based on their relevance to each other. This allows the model to capture longer-range dependencies more effectively.
2. Parallelization: By eliminating recurrency, the Transformer enables much faster parallelization across input sequences, leading to significant
speedups over RNNs.
3. Multi-Head Attention: The Transformer employs multiple attention heads simultaneously, allowing it to focus on different aspects of the input
sequence simultaneously.
4. Position-wise Feedforward Networks: Each element in the input sequence receives its own feedforward network, enabling the model to learn complex
non-linear mappings between input and output sequences.
5. Encoder-Decoder Architecture: Unlike RNNs,

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer model enhances training efficiency by utilizing self-
attention mechanisms instead of traditional recurrent neural networks (RNNs) or convolutional neural networks (CNNs). This allows for faster
parallelization during training, reducing the overall training time required. Additionally, the Transformer model uses a smaller number of parameters
compared to other state-of-the-art models, which further contributes to improved training efficiency.


Sources:
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Attention in Transformers refers to a mechanism that enables the model to focus on specific parts
of the input when generating output. It is used in both the encoder and decoder components of the Transformer architecture. In the encoder, attention
helps the model understand the relationships between different words in a sentence by computing weights for each word based on its relevance to the
current output. In the decoder, attention allows the model to selectively attend to different parts of the input sequence when generating each output
element. This allows the model to generate output sequences that accurately reflect the intended meaning of the input.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer uses several regularization techniques to improve its
performance and prevent overfitting. These include:

1. Dropout: This technique randomly sets a portion of the model's neurons to zero during training, effectively creating an ensemble of different sub-
networks. By doing so, dropout helps to prevent co-adaptation between neighboring neurons and improves generalization.
2. Residual Dropout: Similar to dropout, residual dropout applies dropout to the output of each sub-layer before adding it to the sub-layer input and
normalizing it. This helps to prevent co-adaptation within individual sub-layers.
3. Multi-Head Attention: The Transformer uses multi-head attention mechanisms, which allow multiple attention heads to attend simultaneously to
different aspects of the input sequence. This helps to capture longer-range dependencies and improve overall performance.
4. Positional Encoding: The Transformer uses positional encoding to preserve the order of the input sequence

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer model improved machine translation accuracy by
utilizing self-attention mechanisms instead of traditional recurrent neural networks (RNNs). This allowed for parallelization across input sequences,
reducing training times and increasing efficiency. Additionally, the model incorporated positional encoding in both the encoder and decoder stacks,
further enhancing its performance. By employing label smoothing during training, the model learned to be more uncertain, leading to improved accuracy
and BLEU scores compared to previous models. Overall, the Transformer model demonstrated significant improvements in machine translation quality while
offering faster training times and reduced computational resources.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
  Yes, the Transformer model can be applied beyond machine
translation. In fact, its attention mechanism ma

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What innovation does the Transformer model introduce?
 response:   The Transformer model introduces several innovative features compared to
traditional recurrent neural network (RNN) architectures:

1. Self-Attention Mechanism: Instead of relying solely on recurrence, the Transformer uses a self-attention mechanism to compute the weighted sum of
input elements based on their relevance to each other. This allows the model to capture longer-range dependencies more effectively.
2. Parallelization: By eliminating recurrency, the Transformer enables much faster parallelization across input sequences, leading to significant
speedups over RNNs.
3. Multi-Head Attention: The Transformer employs multiple attention heads simultaneously, allowing it to focus on different aspects of the input
sequence simultaneously.
4. Position-wise Feedforward Networks: Each element in the input sequence receives its own feedforward network, enabling the model to learn complex
non-linear mappings between inpu

In [ ]:
json_file_name = 'AttQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AttQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Coherent Spin Questions

In [ ]:
# Coherent Spin Questions

questions_file_path = '/content/questions/CoSpinQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The objective of utilizing the Heisenberg exchange
interaction in quantum computing is to manipulate and transfer quantum states between particles. However, this concept is often misunderstood due to
incorrect associations with classical mechanical characteristics. In reality, the Heisenberg exchange interaction plays a unique role in quantum
mechanics, unrelated to classical concepts like temperature and mechanical forces.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  There are no external
factors influencing the Heisenberg exchange interaction during quantum spin-state transfer because there are no external factors affecting the
internal quantum state of the particles involved in the transfer.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Quantum scalability is not directly addressed
in discussions of coherent spin-state transfer via the Heisenberg exchange interaction. However, the topic of scaling up quantum technologies,
including quantum computing and information processing, has been extensively studied in various fields. One key aspect of scaling up quantum
technologies is ensuring their reliance on robust and fault-tolerant quantum algorithms, such as those based on coherent spin-state transfer. These
algorithms must be able to operate effectively at larger scales while maintaining their efficiency and accuracy.

To achieve this goal, researchers have developed several strategies aimed at improving the scalability of quantum technologies. For instance, they
have investigated ways to optimize the design of quantum hardware components, such as qubits and quantum gates, to reduce their susceptibility to
decoherence and errors. Additionally, they have explored techniques for enhancing the connectivity and interopera

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Heisenberg exchange plays no direct role in
quantum computing applications. It is an essential component of quantum mechanics but cannot directly manipulate or control quantum states. Instead,
other mechanisms, such as entanglement and quantum teleportation, are used to manage and manipulate quantum information in quantum computing.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  In this article, there are several metaphors used to
explain the initialization of quantum systems. One example is comparing it to loading a spring in classical mechanics. Another metaphor is likening
the process to allowing what is believed to be the Heisenberg exchange interaction to act between the particles, supplemented by mythical
electromagnetic and strong nuclear forces. However, these comparisons are misguiding because they confuse classical mechanical characteristics with
quantum processes.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/quantum.pdf
  Quantum mechanics research is an active field with ongoing
discoveries and advancements. Some potential areas of future development include:

1. Improved understanding of quantum measurement and control: Developing techniques to manipulate and measure quantum systems more precisely will lead
to breakthroughs in fi

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the objective of using Heisenberg exchange in quantum computing?
 response:   The objective of utilizing the Heisenberg exchange
interaction in quantum computing is to manipulate and transfer quantum states between particles. However, this concept is often misunderstood due to
incorrect associations with classical mechanical characteristics. In reality, the Heisenberg exchange interaction plays a unique role in quantum
mechanics, unrelated to classical concepts like temperature and mechanical forces.


Q1: What external factors influence the Heisenberg exchange interaction during quantum spin-state transfer?
 response:   There are no external
factors influencing the Heisenberg exchange interaction during quantum spin-state transfer because there are no external factors affecting the
internal quantum state of the particles involved in the transfer.


Q2: How does coherent spin-state transfer contribute to advancements in quantum technology?
 response:   Coherent spin-state t

In [ ]:
json_file_name = 'CoSpinQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'CoSpinQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)